trying to replicate a system that results in different scores
because of 

In [1]:
import bw2data as bd
print(bd.__version__)

import bw2calc as bc
print(bc.__version__)

import bw2io as bi
print(bi.__version__)

import bw_timex
print(bw_timex.__version__)

from datetime import datetime
from bw_timex import TimexLCA

(4, 0, 'dev46')
2.0.DEV23
0.9.DEV31
0.3.0


## orig system from tests

In [2]:
bd.projects.set_current("__test_nonunitary__")
bd.Database("bio").write(
    {
        ("bio", "CO2"): {
            "type": "emission",
            "name": "carbon dioxide",
        },
    },
)
bd.Database("db_2020").write(
    {
        ("db_2020", "C"): {
            "name": "c",
            "location": "somewhere",
            "reference product": "c",
            "exchanges": [
                {
                    "amount": 3,  # produces a non unitary (not 1) amount
                    "type": "production",
                    "input": ("db_2020", "C"),
                },
                {
                    "amount": 0.5,
                    "type": "biosphere",
                    "input": ("bio", "CO2"),
                },
            ],
        },
    }
)

bd.Database("foreground").write(
    {
        ("foreground", "A"): {
            "name": "a",
            "location": "somewhere",
            "reference product": "a",
            "exchanges": [
                {
                    "amount": 1,
                    "type": "production",
                    "input": ("foreground", "A"),
                },
                {
                    "amount": 1.5,
                    "type": "technosphere",
                    "input": ("db_2020", "C"),
                },
                {
                    "amount": 4,
                    "type": "technosphere",
                    "input": ("foreground", "B"),
                },
            ],
        },
        ("foreground", "B"): {
            "name": "b",
            "location": "somewhere",
            "reference product": "b",
            "exchanges": [
                {
                    "amount": 7,  # produces a non unitary (not 1) amount
                    "type": "production",
                    "input": ("foreground", "B"),
                },
                {
                    "amount": 0.9,
                    "type": "biosphere",
                    "input": ("bio", "CO2"),
                },
                
            ],
        },
    }
)

bd.Method(("GWP", "example")).write(
    [
        (("bio", "CO2"), 1),
    ]
)


100%|██████████| 1/1 [00:00<?, ?it/s]

Vacuuming database 


Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 1/1 [00:00<?, ?it/s]


Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 2/2 [00:00<00:00, 14027.77it/s]

Vacuuming database 


In [3]:
bd.projects.set_current("__test_nonunitary__")

node_a = bd.get_node(database="foreground", code="A")

database_dates = {
    "db_2020": datetime.strptime("2020", "%Y"), #one or two dbs don't make a difference
    "db_2020": datetime.strptime("2030", "%Y"),
    "foreground": "dynamic",
}

tlca = TimexLCA(
    demand={node_a: 1},
    method=("GWP", "example"),
    database_dates=database_dates,
)

tlca.build_timeline(
    starting_datetime=datetime.strptime("2024-01-02", "%Y-%m-%d"),
)



2025-03-21 12:12:39.264 | INFO     | bw_timex.timex_lca:build_timeline:217 - No edge filter function provided. Skipping all edges in background databases.
2025-03-21 12:12:39.314 | INFO     | bw_timex.timeline_builder:get_weights_for_interpolation_between_nearest_years:520 - Reference date 2024-01-01 00:00:00 is lower than all provided dates. Data will be taken from the closest higher year.


Starting graph traversal
Calculation count: 2


2025-03-21 12:12:39.315 | INFO     | bw_timex.timeline_builder:get_weights_for_interpolation_between_nearest_years:520 - Reference date 2024-01-01 00:00:00 is lower than all provided dates. Data will be taken from the closest higher year.
2025-03-21 12:12:39.316 | INFO     | bw_timex.timeline_builder:get_weights_for_interpolation_between_nearest_years:520 - Reference date 2024-01-01 00:00:00 is lower than all provided dates. Data will be taken from the closest higher year.


,date_producer,producer_name,date_consumer,consumer_name,amount,temporal_market_shares
0,2024-01-01,c,2024-01-01,a,1.5,{'db_2020': 1}
1,2024-01-01,a,2024-01-01,-1,1.0,None
2,2024-01-01,b,2024-01-01,a,4.0,None


In [4]:
tlca.lci()
tlca.static_lcia()
print(node_a)
print(tlca.static_score)
print(tlca.base_lca.score)
print(tlca.base_lca.score/tlca.static_score)

'a' (None, somewhere, None)
0.7642857142857142
0.7642857006617955
0.9999999821743119


c:\Users\MULLERA\AppData\Local\anaconda3\envs\test_env1\Lib\site-packages\scipy\sparse\linalg\_dsolve\linsolve.py:603: SparseEfficiencyWarning: splu converted its input to CSC format
  return splu(A).solve


In [5]:
dyn_tech = tlca.create_labelled_technosphere_dataframe()
dyn_tech

,"((db_2020, C), 2030)","((foreground, A), dynamic)","((foreground, B), dynamic)","((temporalized, C), 2024)","((temporalized, A), 2024)","((temporalized, B), 2024)"
"((db_2020, C), 2030)",3.0,-1.5,0.0,-1.0,0.0,0.0
"((foreground, A), dynamic)",0.0,1.0,0.0,0.0,0.0,0.0
"((foreground, B), dynamic)",0.0,-4.0,7.0,0.0,0.0,0.0
"((temporalized, C), 2024)",0.0,0.0,0.0,1.0,-1.5,0.0
"((temporalized, A), 2024)",0.0,0.0,0.0,0.0,1.0,0.0
"((temporalized, B), 2024)",0.0,0.0,0.0,0.0,-4.0,7.0


In [6]:
dyn_bio = tlca.create_labelled_dynamic_biosphere_dataframe()
dyn_bio

,"((db_2020, C), 2030)","((foreground, A), dynamic)","((foreground, B), dynamic)","((temporalized, C), 2024)","((temporalized, A), 2024)","((temporalized, B), 2024)"
"(128, 2024-01-01T00:00:00)",0.0,0.0,0.0,0.166667,0.0,0.9


## mod system 1

In [7]:
from bw_temporalis import TemporalDistribution
import numpy as np
bd.projects.set_current("__test_nonunitary_mod__")
bd.Database("bio").write(
    {
        ("bio", "CO2"): {
            "type": "emission",
            "name": "carbon dioxide",
        },
    },
)
bd.Database("db_2020").write(
    {
        ("db_2020", "C"): {
            "name": "c",
            "location": "somewhere",
            "reference product": "c",
            "exchanges": [
                {
                    "amount": 3,  # produces a non unitary (not 1) amount
                    "type": "production",
                    "input": ("db_2020", "C"),
                },
                {
                    "amount": 0.5,
                    "type": "biosphere",
                    "input": ("bio", "CO2"),
                },
            ],
        },
    }
)

bd.Database("foreground").write(
    {
        ("foreground", "A"): {
            "name": "a",
            "location": "somewhere",
            "reference product": "a",
            "exchanges": [
                {
                    "amount":-0.67,
                    "type": "production",
                    "input": ("foreground", "A"),
                },
                {
                    "amount": 1.5,
                    "type": "technosphere",
                    "input": ("db_2020", "C"),
                },
                {
                    "amount": -3,
                    "type": "technosphere",
                    "input": ("foreground", "B"),
                    "temporal_distribution": TemporalDistribution(
                            np.array([-24, -12], dtype="timedelta64[M]"),
                            np.array([0.5, 0.5]),
                        ),
                },
                {
                    "amount": 3.5,
                    "type": "biosphere",
                    "input": ("bio", "CO2"),
                },
            ],
        },
        ("foreground", "B"): {
            "name": "b",
            "location": "somewhere",
            "reference product": "b",
            "exchanges": [
                {
                    "amount": -12,  # produces a non unitary (not 1) amount
                    "type": "production",
                    "input": ("foreground", "B"),
                },
                {
                    "amount": 6,
                    "type": "technosphere",
                    "input": ("db_2020", "C"),
                },
                {
                    "amount":-34,
                    "type": "biosphere",
                    "input": ("bio", "CO2"),
                },
            ],
        },
    }
)

bd.Method(("GWP", "example")).write(
    [
        (("bio", "CO2"), 1),
    ]
)



100%|██████████| 1/1 [00:00<?, ?it/s]


Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 1/1 [00:00<00:00, 1999.19it/s]


Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 2/2 [00:00<?, ?it/s]

Vacuuming database 


In [8]:
bd.projects.set_current("__test_nonunitary_mod__")

node_a = bd.get_node(database="foreground", code="A")

database_dates = {
    "db_2020": datetime.strptime("2020", "%Y"), #one or two dbs don't make a difference
    "db_2020": datetime.strptime("2030", "%Y"),
    "foreground": "dynamic",
}

tlca = TimexLCA(
    demand={node_a: 1},
    method=("GWP", "example"),
    database_dates=database_dates,
)

tlca.build_timeline(
    starting_datetime=datetime.strptime("2024-01-02", "%Y-%m-%d"),
)




2025-03-21 12:12:40.208 | INFO     | bw_timex.timex_lca:build_timeline:217 - No edge filter function provided. Skipping all edges in background databases.
2025-03-21 12:12:40.271 | INFO     | bw_timex.timeline_builder:get_weights_for_interpolation_between_nearest_years:520 - Reference date 2022-01-01 00:00:00 is lower than all provided dates. Data will be taken from the closest higher year.
2025-03-21 12:12:40.272 | INFO     | bw_timex.timeline_builder:get_weights_for_interpolation_between_nearest_years:520 - Reference date 2022-01-01 00:00:00 is lower than all provided dates. Data will be taken from the closest higher year.
2025-03-21 12:12:40.274 | INFO     | bw_timex.timeline_builder:get_weights_for_interpolation_between_nearest_years:520 - Reference date 2023-01-01 00:00:00 is lower than all provided dates. Data will be taken from the closest higher year.
2025-03-21 12:12:40.275 | INFO     | bw_timex.timeline_builder:get_weights_for_interpolation_between_nearest_years:520 - Referen

Starting graph traversal
Calculation count: 3


,date_producer,producer_name,date_consumer,consumer_name,amount,temporal_market_shares
0,2022-01-01,c,2022-01-01,b,0.5,{'db_2020': 1}
1,2022-01-01,b,2024-01-01,a,-2.238806,None
2,2023-01-01,c,2023-01-01,b,0.5,{'db_2020': 1}
3,2023-01-01,b,2024-01-01,a,-2.238806,None
4,2024-01-01,c,2024-01-01,a,2.238806,{'db_2020': 1}
5,2024-01-01,a,2024-01-01,-1,1.0,None


In [9]:
tlca.lci()
tlca.static_lcia()
print(node_a)
print(tlca.static_score)
print(tlca.base_lca.score)
print(tlca.base_lca.score/tlca.static_score)

'a' (None, somewhere, None)
6.716417613022053
6.7164177431458
1.0000000193739809


c:\Users\MULLERA\AppData\Local\anaconda3\envs\test_env1\Lib\site-packages\scipy\sparse\linalg\_dsolve\linsolve.py:603: SparseEfficiencyWarning: splu converted its input to CSC format
  return splu(A).solve


In [10]:
tlca.create_labelled_technosphere_dataframe()

,"((db_2020, C), 2030)","((foreground, A), dynamic)","((foreground, B), dynamic)","((temporalized, C), 2022)","((temporalized, B), 2022)","((temporalized, C), 2023)","((temporalized, B), 2023)","((temporalized, C), 2024)","((temporalized, A), 2024)"
"((db_2020, C), 2030)",3.0,-1.50,-6.0,-1.0,0.0,-1.0,0.0,-1.0,0.00
"((foreground, A), dynamic)",0.0,-0.67,0.0,0.0,0.0,0.0,0.0,0.0,0.00
"((foreground, B), dynamic)",0.0,3.00,-12.0,0.0,0.0,0.0,0.0,0.0,0.00
"((temporalized, C), 2022)",0.0,0.00,0.0,1.0,-6.0,0.0,0.0,0.0,0.00
"((temporalized, B), 2022)",0.0,0.00,0.0,0.0,-12.0,0.0,0.0,0.0,1.50
"((temporalized, C), 2023)",0.0,0.00,0.0,0.0,0.0,1.0,-6.0,0.0,0.00
"((temporalized, B), 2023)",0.0,0.00,0.0,0.0,0.0,0.0,-12.0,0.0,1.50
"((temporalized, C), 2024)",0.0,0.00,0.0,0.0,0.0,0.0,0.0,1.0,-1.50
"((temporalized, A), 2024)",0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,-0.67


In [11]:
tlca.create_labelled_biosphere_dataframe()

,"((db_2020, C), 2030)","((foreground, A), dynamic)","((foreground, B), dynamic)","((temporalized, C), 2022)","((temporalized, B), 2022)","((temporalized, C), 2023)","((temporalized, B), 2023)","((temporalized, C), 2024)","((temporalized, A), 2024)"
"(bio, CO2)",0.5,3.5,-34.0,0.0,-34.0,0.0,-34.0,0.0,3.5


## mod sys 2

In [12]:
bd.projects.set_current("__test_nonunitary_mod2__")
bd.Database("bio").write(
    {
        ("bio", "CO2"): {
            "type": "emission",
            "name": "carbon dioxide",
        },
    },
)
bd.Database("db_2020").write(
    {
        ("db_2020", "C"): {
            "name": "c",
            "location": "somewhere",
            "reference product": "c",
            "exchanges": [
                {
                    "amount": 3,  # produces a non unitary (not 1) amount
                    "type": "production",
                    "input": ("db_2020", "C"),
                },
                {
                    "amount": 0.5,
                    "type": "biosphere",
                    "input": ("bio", "CO2"),
                },
            ],
        },
    }
)

bd.Database("foreground").write(
    {
        ("foreground", "A"): {
            "name": "a",
            "location": "somewhere",
            "reference product": "a",
            "exchanges": [
                {
                    "amount": 1,
                    "type": "production",
                    "input": ("foreground", "A"),
                },
                {
                    "amount": 1.5,
                    "type": "technosphere",
                    "input": ("db_2020", "C"),
                },
                {
                    "amount": 4,
                    "type": "technosphere",
                    "input": ("foreground", "A1"),
                },
                
            ],
            
        },
        ("foreground", "A1"): {
            "name": "a1",
            "location": "somewhere",
            "reference product": "a1",
            "exchanges": [
                {
                    "amount": 2,  # produces a non unitary (not 1) amount
                    "type": "production",
                    "input": ("foreground", "A1"),
                },
                {
                    "amount": 2,
                    "type": "technosphere",
                    "input": ("foreground", "B"),
                },
            ],
        },
        ("foreground", "B"): {
            "name": "b",
            "location": "somewhere",
            "reference product": "b",
            "exchanges": [
                {
                    "amount": 7,  # produces a non unitary (not 1) amount
                    "type": "production",
                    "input": ("foreground", "B"),
                },
                {
                    "amount": 0.9,
                    "type": "biosphere",
                    "input": ("bio", "CO2"),
                },
            ],
        },

    }
)

bd.Method(("GWP", "example")).write(
    [
        (("bio", "CO2"), 1),
    ]
)

100%|██████████| 1/1 [00:00<?, ?it/s]


Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 1/1 [00:00<?, ?it/s]


Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 3/3 [00:00<?, ?it/s]

Vacuuming database 


In [13]:
bd.projects.set_current("__test_nonunitary_mod2__")

node_a = bd.get_node(database="foreground", code="A")

database_dates = {
    "db_2020": datetime.strptime("2020", "%Y"), #one or two dbs don't make a difference
    "db_2020": datetime.strptime("2030", "%Y"),
    "foreground": "dynamic",
}

tlca = TimexLCA(
    demand={node_a: 1},
    method=("GWP", "example"),
    database_dates=database_dates,
)

tlca.build_timeline(
    starting_datetime=datetime.strptime("2024-01-02", "%Y-%m-%d"),
)


2025-03-21 12:12:41.394 | INFO     | bw_timex.timex_lca:build_timeline:217 - No edge filter function provided. Skipping all edges in background databases.
2025-03-21 12:12:41.442 | INFO     | bw_timex.timeline_builder:get_weights_for_interpolation_between_nearest_years:520 - Reference date 2024-01-01 00:00:00 is lower than all provided dates. Data will be taken from the closest higher year.
2025-03-21 12:12:41.443 | INFO     | bw_timex.timeline_builder:get_weights_for_interpolation_between_nearest_years:520 - Reference date 2024-01-01 00:00:00 is lower than all provided dates. Data will be taken from the closest higher year.
2025-03-21 12:12:41.444 | INFO     | bw_timex.timeline_builder:get_weights_for_interpolation_between_nearest_years:520 - Reference date 2024-01-01 00:00:00 is lower than all provided dates. Data will be taken from the closest higher year.
2025-03-21 12:12:41.446 | INFO     | bw_timex.timeline_builder:get_weights_for_interpolation_between_nearest_years:520 - Referen

Starting graph traversal
Calculation count: 3


,date_producer,producer_name,date_consumer,consumer_name,amount,temporal_market_shares
0,2024-01-01,c,2024-01-01,a,1.5,{'db_2020': 1}
1,2024-01-01,a,2024-01-01,-1,1.0,None
2,2024-01-01,a1,2024-01-01,a,4.0,None
3,2024-01-01,b,2024-01-01,a1,1.0,None


In [14]:
tlca.lci()
tlca.static_lcia()
print(node_a)
print(tlca.static_score)
print(tlca.base_lca.score)
print(tlca.base_lca.score/tlca.static_score)

'a' (None, somewhere, None)
0.7642857142857142
0.7642857006617955
0.9999999821743119


c:\Users\MULLERA\AppData\Local\anaconda3\envs\test_env1\Lib\site-packages\scipy\sparse\linalg\_dsolve\linsolve.py:603: SparseEfficiencyWarning: splu converted its input to CSC format
  return splu(A).solve


In [15]:
tlca.create_labelled_technosphere_dataframe()

,"((db_2020, C), 2030)","((foreground, A), dynamic)","((foreground, A1), dynamic)","((foreground, B), dynamic)","((temporalized, C), 2024)","((temporalized, A), 2024)","((temporalized, A1), 2024)","((temporalized, B), 2024)"
"((db_2020, C), 2030)",3.0,-1.5,0.0,0.0,-1.0,0.0,0.0,0.0
"((foreground, A), dynamic)",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
"((foreground, A1), dynamic)",0.0,-4.0,2.0,0.0,0.0,0.0,0.0,0.0
"((foreground, B), dynamic)",0.0,0.0,-2.0,7.0,0.0,0.0,0.0,0.0
"((temporalized, C), 2024)",0.0,0.0,0.0,0.0,1.0,-1.5,0.0,0.0
"((temporalized, A), 2024)",0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
"((temporalized, A1), 2024)",0.0,0.0,0.0,0.0,0.0,-4.0,2.0,0.0
"((temporalized, B), 2024)",0.0,0.0,0.0,0.0,0.0,0.0,-2.0,7.0
